# Create enhanced static stac catalog with added metadata 

Approach:

1. Use Umbra STAC API to find public data + metadata 
1. Find public GeoTiffs in public S3 bucket (unfortunately not linked in API return!)
1. Generate STAC metadata with TiTiler by actually reading GEC file
    - we can trust these footprints + adds other information of interest (proj extension, raster extension)
1. Add custom metadata we are interested in 
    - e.g. processor version, RPCs, sar:observation_direction, etc
1. Merge Umbra metadata with TiTiler-generated metadata
1. Save as a static STAC catalog for easy future referecnce

In [1]:
# Make sure we have an updated list of assets
#!aws s3 ls --no-sign-request s3://umbra-open-data-catalog/ --recursive > umbra-open-data-catalog-list.txt

In [1]:
import os
import asyncio
import urllib.parse
import requests
import rasterio

from pystac.layout import TemplateLayoutStrategy

os.environ['AWS_NO_SIGN_REQUEST'] = 'YES'
os.environ['GDAL_DISABLE_READDIR_ON_OPEN'] = 'EMPTY_DIR'

import geopandas as gpd
import pystac_client
import stac_geoparquet
import pystac

%matplotlib inline

In [2]:
stac_geoparquet.__version__

'0.6.1.dev1+g4b00f5b'

## Search Umbra API

In [3]:
aoi = gpd.read_file('panama-canal.geojson')

In [4]:
# Search for acquisitions in AWS Open Data Catalog
# NOTE: different endpoint, but still need auth
stac_api_url = "https://api.canopy.umbra.space/archive/"
catalog = pystac_client.Client.open(stac_api_url,
                                    headers={"authorization": f"Bearer {os.environ.get('UMBRA_API_TOKEN')}" }
)
catalog

# Hack fix for broken API links (need to be https://)
# https://github.com/huskysar/umbra/issues/1
for link in catalog.get_links():
    link.target = link.target.replace('http://','https://')

In [5]:
limit_results=3000

cql2filter = {
    "op": "=",
    "args": [
      {
        "property": "umbra:open-data-catalog"
      },
      True
    ]
  }

stac_search = catalog.search(
    intersects=aoi.geometry.iloc[0],
    max_items=limit_results,
    limit=limit_results,
    collections=["umbra-sar"],
    filter=cql2filter,
)

items = stac_search.item_collection()
#stac_search.matched() # doesn't work for umbra
len(items)

195

In [6]:
!aws s3 --no-sign-request ls 's3://umbra-open-data-catalog/sar-data/tasks/Panama Canal, Panama/' | wc

     187     374   12903


In [7]:
# Hmmm, so there is a mismatch between the number of items found and the number of files in the bucket
# Apparently some are also under a 'ship detection' folder, and some are simply missing

In [8]:
# STAC Item ID should be only at top level, not under 'properties'
# Still 'valid' according to items[0].validate(), but messes up stac-geoparquet parsing
_ = [i.properties.pop('id', None) for i in items]

# Warning: older items do not have 'created' or 'updated' fields
# STAC geoparquet requires values for all rows in a column, so will add the following to STAC when saved:
#"created": null

In [9]:
record_batch_reader = stac_geoparquet.arrow.parse_stac_items_to_arrow(items)
gf = gpd.GeoDataFrame.from_arrow(record_batch_reader)  # doesn't keep arrow dtypes
gf

,assets,bbox,collection,geometry,id,links,stac_extensions,stac_version,type,created,...,umbra:open-data-catalog,umbra:slant_range_kilometers,umbra:squint_angle_degrees,umbra:squint_angle_degrees_off_broadside,umbra:squint_angle_engineering_degrees,umbra:squint_angle_exploitation_degrees,umbra:target_azimuth_angle_degrees,umbra:task_id,updated,view:incidence_angle
0,{'thumbnail': {'description': 'Low-resolution ...,"{'xmin': -79.60445239153408, 'ymin': 8.9507315...",umbra-sar,"POLYGON Z ((-79.55898 8.9966 14.31562, -79.604...",b0f15df2-f57e-46c2-b59a-2e03181c394f,[{'href': 'http://api.canopy.umbra.space/archi...,[https://stac-extensions.github.io/view/v1.0.0...,1.0.0,Feature,2025-01-20 18:39:07.462998+00:00,...,True,670.504272,160.026749,19.973259,-70.026741,19.973259,97.983406,234aee0f-59a6-4b57-94f2-e799357b5352,2025-01-20 18:39:07.463002+00:00,33.830276
1,{'thumbnail': {'description': 'Low-resolution ...,"{'xmin': -79.60429112089089, 'ymin': 8.9508891...",umbra-sar,"POLYGON Z ((-79.55847 8.9961 14.31548, -79.603...",015fc551-3e2f-4ba4-9069-b9b091601e05,[{'href': 'http://api.canopy.umbra.space/archi...,[https://stac-extensions.github.io/view/v1.0.0...,1.0.0,Feature,2025-01-14 09:14:54.682994+00:00,...,True,758.888184,160.425781,19.574226,-70.425774,19.574226,97.397911,bfe9e972-3bf2-4da4-b9f2-24fd8f54d157,2025-01-14 09:14:54.682999+00:00,43.784485
2,{'thumbnail': {'description': 'Low-resolution ...,"{'xmin': -79.60560086374905, 'ymin': 8.9495896...",umbra-sar,"POLYGON Z ((-79.56023 8.99774 14.31596, -79.60...",23910fdd-69f0-401d-8c84-eea9272cda2d,[{'href': 'http://api.canopy.umbra.space/archi...,[https://stac-extensions.github.io/view/v1.0.0...,1.0.0,Feature,2025-01-13 02:46:10.317781+00:00,...,True,771.610046,164.988754,15.011246,-74.988754,15.011246,92.787834,54ca569e-57d2-446a-bc0d-ba2105519fc2,2025-01-13 02:46:10.317786+00:00,44.914482
3,{'thumbnail': {'description': 'Low-resolution ...,"{'xmin': -79.61299625212686, 'ymin': 8.9422371...",umbra-sar,"POLYGON Z ((-79.54977 8.97951 14.31252, -79.58...",40e39634-c0f3-4907-bb4e-208d8c644cfe,[{'href': 'http://api.canopy.umbra.space/archi...,[https://stac-extensions.github.io/view/v1.0.0...,1.0.0,Feature,2024-11-01 01:42:50.539009+00:00,...,True,570.082581,128.792328,51.207672,-38.792328,51.207672,129.777008,3d260f7b-9e52-45d3-b0d7-8c2f49507459,2024-11-01 01:42:50.539013+00:00,29.597500
4,{'thumbnail': {'description': 'Low-resolution ...,"{'xmin': -79.61351789024071, 'ymin': 8.9417192...",umbra-sar,"POLYGON Z ((-79.54925 8.97479 14.31231, -79.58...",52f2317f-091b-4f90-b385-08c93655e089,[{'href': 'http://api.canopy.umbra.space/archi...,[https://stac-extensions.github.io/view/v1.0.0...,1.0.0,Feature,2024-09-10 10:00:00.425293+00:00,...,True,567.743591,120.062416,59.937584,-30.062416,59.937584,138.559494,ba1ca3b0-f458-4cd9-8e99-52d2d899d5dd,2024-09-10 10:00:00.425300+00:00,23.750572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,{'thumbnail': {'description': '512x512 PNG thu...,"{'xmin': -79.60278533117051, 'ymin': 8.9523874...",umbra-sar,"POLYGON Z ((-79.59565 8.99495 0, -79.60279 8.9...",e022c48b-7374-48bd-953b-39fd96e08eb8,[{'href': 'http://api.canopy.umbra.space/archi...,[https://stac-extensions.github.io/view/v1.0.0...,1.0.0,Feature,NaT,...,True,925.267761,181.298401,1.298401,-91.298401,-1.298401,281.031433,24f8952c-35cb-47bc-82f9-5a651255c282,NaT,58.990635
191,{'thumbnail': {'description': '512x512 PNG thu...,"{'xmin': -79.6023803135697, 'ymin': 8.95279180...",umbra-sar,"POLYGON Z ((-79.60238 8.98844 0, -79.59624 8.9...",41215873-ff67-4690-b34c-a898037865dd,[{'href': 'http://api.canopy.umbra.space/archi...,[https://stac-extensions.github.io/view/v1.0.0...,1.0.0,Feature,NaT,...,True,641.910400,0.153363,0.153363,89.846637,0.153363,258.976776,2e249b81-1b12-4d6c-8aa7-9a0b84554afe,NaT,36.410610
192,{'thumbnail': {'description': '512x512 PNG thu...,"{'xmin': -79.60220489144645, 'ymin': 8.9529645...",umbra-sar,"POLYGON Z ((-79.59648 8.99437

## Link to public GeoTiff Assets

In [10]:

def s3_to_http(s3_path):
    # NOTE: titiler requires https:// links
    #http://umbra-open-data-catalog.s3.amazonaws.com/sar-data/tasks/Panama%20Canal%2C%20Panama/fac35699-2f8d-4c28-ab5e-638182373f34/2024-02-17-15-00-05_UMBRA-04/2024-02-17-15-00-05_UMBRA-04_GEC.tif
    url = s3_path.replace('s3://umbra-open-data-catalog/','https://umbra-open-data-catalog.s3.amazonaws.com/')
    sanitized = urllib.parse.quote(url, safe=':/')
    return sanitized

def get_asset_hrefs(task_id, prefix='s3://umbra-open-data-catalog/sar-data'):
    """ extract hrefs for a given umbra:task_id from s3 bucket listing (umbra-open-data-catalog-list.txt) """
    with open('umbra-open-data-catalog-list.txt') as f:
        lines = f.readlines()
    assets = [x.rstrip() for x in lines if task_id in x]
    asset_paths = [prefix + x.split('sar-data')[1] for x in assets]
    return asset_paths


def make_asset_dictionary(asset_paths):
    """ assign href based on file name suffix """
    # NOTE: newer versions will switch to MM.tif
    asset_map = {}
    for asset in asset_paths:
        if asset.endswith('GEC.tif') or asset.endswith('MM.tif'):
            asset_map['gec'] = {
    "description": "MONOSTATIC TIFF",
    "href": s3_to_http(asset),
    "roles": [
        "data"
    ],
    "title": "TIFF",
    "type": "image/tiff; application=geotiff; profile=cloud-optimized"
}
        elif asset.endswith('METADATA.json'):
            asset_map['metadata'] = {
    "description": "MONOSTATIC METADATA",
    "href": s3_to_http(asset),
    "roles": [
        "metadata"
    ],
    "title": "METADATA",
    "type": "application/json"
}
        elif asset.endswith('SICD.nitf'):
            asset_map['sicd'] = {
    "description": "MONOSTATIC SICD",
    "href": s3_to_http(asset),
    "roles": [
        "data"
    ],
    "title": "SICD",
    "type": "application/octet-stream"
}
# Some collects missing SIDD. Need all items to have same assets, so just skip for now
# I don't think we need it, and can just replace SICD with SIDD in path if is needed
#         elif asset.endswith('SIDD.nitf'):
#             asset_map['sidd'] = {
#     "description": "MONOSTATIC SIDD",
#     "href": asset,
#     #or how stac-geoparquet does it: np.array(['data'], dtype=object)
#     "roles": [
#         "data"
#     ],
#     "title": "SIDD",
#     "type": "application/octet-stream"
# }

    return asset_map

In [11]:
i = 10 #'234aee0f-59a6-4b57-94f2-e799357b5352' not in there yet!
hrefs = get_asset_hrefs(gf['umbra:task_id'].iloc[10])
make_asset_dictionary(hrefs)

{'gec': {'description': 'MONOSTATIC TIFF',
  'href': 'https://umbra-open-data-catalog.s3.amazonaws.com/sar-data/tasks/Panama%20Canal%2C%20Panama/81a45379-27f5-4859-b333-b1c9e3d96dcc/2024-08-20-02-38-52_UMBRA-05/2024-08-20-02-38-52_UMBRA-05_GEC.tif',
  'roles': ['data'],
  'title': 'TIFF',
  'type': 'image/tiff; application=geotiff; profile=cloud-optimized'},
 'metadata': {'description': 'MONOSTATIC METADATA',
  'href': 'https://umbra-open-data-catalog.s3.amazonaws.com/sar-data/tasks/Panama%20Canal%2C%20Panama/81a45379-27f5-4859-b333-b1c9e3d96dcc/2024-08-20-02-38-52_UMBRA-05/2024-08-20-02-38-52_UMBRA-05_METADATA.json',
  'roles': ['metadata'],
  'title': 'METADATA',
  'type': 'application/json'},
 'sicd': {'description': 'MONOSTATIC SICD',
  'href': 'https://umbra-open-data-catalog.s3.amazonaws.com/sar-data/tasks/Panama%20Canal%2C%20Panama/81a45379-27f5-4859-b333-b1c9e3d96dcc/2024-08-20-02-38-52_UMBRA-05/2024-08-20-02-38-52_UMBRA-05_SICD.nitf',
  'roles': ['data'],
  'title': 'SICD',
  

In [12]:
# Add Assets
gf['assets'] = gf['umbra:task_id'].apply(lambda x: make_asset_dictionary(get_asset_hrefs(x)))

In [13]:
# Drop rows with empty assets
gf = gf[gf['assets'].apply(lambda x: len(x) > 0)].reset_index(drop=True)
len(gf)

185

In [14]:
gf.assets.iloc[0]

{'gec': {'description': 'MONOSTATIC TIFF',
  'href': 'https://umbra-open-data-catalog.s3.amazonaws.com/sar-data/tasks/Panama%20Canal%2C%20Panama/bfe9e972-3bf2-4da4-b9f2-24fd8f54d157/2025-01-13-03-36-19_UMBRA-09/2025-01-13-03-36-19_UMBRA-09_GEC.tif',
  'roles': ['data'],
  'title': 'TIFF',
  'type': 'image/tiff; application=geotiff; profile=cloud-optimized'},
 'metadata': {'description': 'MONOSTATIC METADATA',
  'href': 'https://umbra-open-data-catalog.s3.amazonaws.com/sar-data/tasks/Panama%20Canal%2C%20Panama/bfe9e972-3bf2-4da4-b9f2-24fd8f54d157/2025-01-13-03-36-19_UMBRA-09/2025-01-13-03-36-19_UMBRA-09_METADATA.json',
  'roles': ['metadata'],
  'title': 'METADATA',
  'type': 'application/json'},
 'sicd': {'description': 'MONOSTATIC SICD',
  'href': 'https://umbra-open-data-catalog.s3.amazonaws.com/sar-data/tasks/Panama%20Canal%2C%20Panama/bfe9e972-3bf2-4da4-b9f2-24fd8f54d157/2025-01-13-03-36-19_UMBRA-09/2025-01-13-03-36-19_UMBRA-09_SICD.nitf',
  'roles': ['data'],
  'title': 'SICD',
  

In [15]:
# Convert Dataframe back to stac items
batch = stac_geoparquet.arrow.stac_table_to_items(gf.to_arrow())
items = [pystac.Item.from_dict(x) for x in batch]

## Read Additional Metadata from Tifs and Metadata Files

In [16]:
# Format for TiTiler
def get_datetime(row):
    start = row.start_datetime.isoformat()
    end = row.end_datetime.isoformat()
    datestr = f'{start}/{end}'
    #print(datestr)
    return datestr

get_datetime(gf.iloc[0])

'2025-01-13T03:36:20+00:00/2025-01-13T03:36:34.218407+00:00'

In [17]:
#s = gf1.iloc[0]
#print(s.start_datetime, s.end_datetime)
#(s.end_datetime - s.start_datetime).seconds
gf['huskysar:duration'] = gf.apply(lambda row: (row.end_datetime - row.start_datetime).seconds, axis=1)

In [18]:
# Format for TiTiler
def get_duration(row):
    start = row.start_datetime.isoformat()
    end = row.end_datetime.isoformat()
    datestr = f'{start}/{end}'
    #print(datestr)
    return datestr

get_datetime(gf.iloc[0])

'2025-01-13T03:36:20+00:00/2025-01-13T03:36:34.218407+00:00'

In [19]:
# Change itemIDs to GEC Tif names
for i in items:
    i.properties['umbra:stac_id'] = i.id
    i.id = i.assets['gec'].href.split('/')[-1][:-4]

In [20]:
# Read GeoTIFF Metadata

def get_gtiff_metadata(URL):
    with rasterio.open(URL) as src:
        gtiff_metadata = src.tags()
        if src.rpcs is not None:
            has_rpcs = True
        else:
            has_rpcs = False
        return gtiff_metadata.get('PROCESSOR'), has_rpcs

get_gtiff_metadata(URL = gf.iloc[10].assets['gec']['href'])

('3.48.0', True)

In [21]:
def read_metadata(url):
    r = requests.get(url)
    return r.json()

meta = read_metadata(gf.iloc[10].assets['metadata']['href'])

In [22]:
meta['collects'][0]['sceneSize']

'5x5_KM'

In [23]:
def get_orbit_state_and_observation_direction(url):
    r = requests.get(url)
    data = r.json()
    orbit_state =  data['collects'][0].get('satelliteTrack').lower()
    obs_dir = data['collects'][0].get('observationDirection').lower()
    scene_size = data['collects'][0].get('sceneSize')
    return orbit_state, obs_dir, scene_size

get_orbit_state_and_observation_direction(gf.iloc[10].assets['metadata']['href'])

('ascending', 'right', '5x5_KM')

In [24]:
metadata_urls = gf['assets'].apply(lambda x: x['metadata']['href']).values
metadata_urls[:3]

array(['https://umbra-open-data-catalog.s3.amazonaws.com/sar-data/tasks/Panama%20Canal%2C%20Panama/bfe9e972-3bf2-4da4-b9f2-24fd8f54d157/2025-01-13-03-36-19_UMBRA-09/2025-01-13-03-36-19_UMBRA-09_METADATA.json',
       'https://umbra-open-data-catalog.s3.amazonaws.com/sar-data/tasks/Panama%20Canal%2C%20Panama/54ca569e-57d2-446a-bc0d-ba2105519fc2/2025-01-12-03-35-18_UMBRA-09/2025-01-12-03-35-18_UMBRA-09_METADATA.json',
       'https://umbra-open-data-catalog.s3.amazonaws.com/sar-data/tasks/Panama%20Canal%2C%20Panama/3d260f7b-9e52-45d3-b0d7-8c2f49507459/2024-10-31-03-29-59_UMBRA-08/2024-10-31-03-29-59_UMBRA-08_METADATA.json'],
      dtype=object)

In [25]:

# Neat: How to wrap a synchronous function to run asynchronously!
# https://www.youtube.com/watch?v=p8tnmEdeOU0
async def get_obs_dir_async(url):
    response = await asyncio.to_thread(get_orbit_state_and_observation_direction, url)
    return response

extracted_metadata = await asyncio.gather(*[get_obs_dir_async(url) for url in metadata_urls])

In [26]:
# Same to get all the processor and has_rpcs
# KeyError: 'PROCESSOR'
tif_urls = gf['assets'].apply(lambda x: x['gec']['href']).values
tif_urls[:3]

async def get_tiff_info_async(url):
    response = await asyncio.to_thread(get_gtiff_metadata, url)
    return response

extracted_tifftags = await asyncio.gather(*[get_tiff_info_async(url) for url in tif_urls])

In [27]:
gf['sar:observation_direction'] = [x[1] for x in extracted_metadata]
gf['sat:orbit_state'] = [x[0] for x in extracted_metadata]
gf['huskysar:scene_size'] = [x[2] for x in extracted_metadata]
gf['umbra:processor'] = [x[0] for x in extracted_tifftags]
gf['huskysar:rpcs'] = [x[1] for x in extracted_tifftags]

In [28]:
gf.iloc[0]

assets                                      {'gec': {'description': 'MONOSTATIC TIFF', 'hr...
bbox                                        {'xmin': -79.60429112089089, 'ymin': 8.9508891...
collection                                                                          umbra-sar
geometry                                    POLYGON Z ((-79.55847308842013 8.9960964929908...
id                                                       015fc551-3e2f-4ba4-9069-b9b091601e05
links                                       [{'href': 'http://api.canopy.umbra.space/archi...
stac_extensions                             [https://stac-extensions.github.io/view/v1.0.0...
stac_version                                                                            1.0.0
type                                                                                  Feature
created                                                      2025-01-14 09:14:54.682994+00:00
datetime                                                    

In [29]:
# For now, only work with data that has RPCs
gf1 = gf[gf['huskysar:rpcs']].reset_index(drop=True)
len(gf1)

76

In [30]:
# recent processors switched to 5x5 instead of 4x4 default scene size
gf1['huskysar:scene_size'].value_counts()

huskysar:scene_size
5x5_KM    76
Name: count, dtype: int64

In [31]:
# RPCs after 2024-02-01 processor>3.41.0
gf1.start_datetime.min(), gf1.end_datetime.max()

(Timestamp('2024-02-01 03:28:14+0000', tz='UTC'),
 Timestamp('2025-01-13 03:36:34.218407+0000', tz='UTC'))

In [32]:
# Convert Dataframe back to stac items
batch = stac_geoparquet.arrow.stac_table_to_items(gf1.to_arrow())
orig_items = [pystac.Item.from_dict(x) for x in batch]
len(orig_items)

76

## Use TiTiler to generate STAC metadata directly from TIFs

In [33]:
i = 0
URL = gf.iloc[i].assets['gec']['href']
ID = URL.split('/')[-1][:-4]
DATETIME = get_datetime(gf.iloc[i])

# https://titiler.xyz/api.html#/
# r = requests.get("https://titiler.xyz/cog/stac?asset_name=data&asset_media_type=auto&with_proj=true&with_raster=true&with_eo=true&max_size=1024&geometry_densify=0&geometry_precision=-1&url=http%3A%2F%2Fumbra-open-data-catalog.s3.amazonaws.com%2Fsar-data%2Ftasks%2FPanama%2520Canal%252C%2520Panama%2Fbfe9e972-3bf2-4da4-b9f2-24fd8f54d157%2F2025-01-13-03-36-19_UMBRA-09%2F2025-01-13-03-36-19_UMBRA-09_GEC.tif")
baseurl = "https://titiler.xyz/cog/stac"
params = {"id": ID,
    "asset_name": "data",
            "collection": "umbra-sar",
            "datetime": DATETIME,
            "asset_media_type": "auto",
            "with_proj": "true",
            "with_raster": "true",
            "with_eo": "true",
            #"max_size": "1024",
            "geometry_densify": "0",
            "geometry_precision": "-1",
            "url": URL}
r = requests.get(baseurl, params=params)
r.json()

{'type': 'Feature',
 'stac_version': '1.0.0',
 'stac_extensions': ['https://stac-extensions.github.io/projection/v1.1.0/schema.json',
  'https://stac-extensions.github.io/raster/v1.1.0/schema.json',
  'https://stac-extensions.github.io/eo/v1.1.0/schema.json'],
 'id': '2025-01-13-03-36-19_UMBRA-09_GEC',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-79.60429181215505, 8.950889800680791],
    [-79.55847377933229, 8.950889800680791],
    [-79.55847377933229, 8.99644461237005],
    [-79.60429181215505, 8.99644461237005],
    [-79.60429181215505, 8.950889800680791]]]},
 'bbox': [-79.60429181215505,
  8.950889800680791,
  -79.55847377933229,
  8.99644461237005],
 'properties': {'start_datetime': '2025-01-13T03:36:20Z',
  'end_datetime': '2025-01-13T03:36:34.218407Z',
  'proj:epsg': 4326,
  'proj:geometry': {'type': 'Polygon',
   'coordinates': [[[-79.60429181215505, 8.950889800680791],
     [-79.55847377933229, 8.950889800680791],
     [-79.55847377933229, 8.99644461237005],
     [-79

#### How does the above compare to UMBRA API STAC?

```
#items[0].to_dict() # unathorized!
#gf.iloc[0].to_dict()
```

A few observations:

1. Umbra uses POLYGONZ (with heights (from where?...))
 'geometry': <POLYGON Z ((-79.558 8.996 14.315, -79.604 8.996 14.316, -79.604 8.951 14.31...>,
1. Umbra bbox approx equivalent (given 5 decimals ~ +/- 1m 6-> +/-cm)
s = [-79.60429181215505, 8.950889800680791, -79.55847377933229, 8.99644461237005],
u = [-79.60429112089089, 8.950889110300890, -79.55847308842013, 8.996443921817997]



In [34]:
# Generate STAC with TiTiler (actually read the GeoTiff)
# This will add additional detail like 'proj' and 'raster' extension information &datetime={datetime} id={id} collection={collection}&asset_name={asset_name}&asset_roles=data&
def create_stac_item(row):
    baseurl = "https://titiler.xyz/cog/stac"

    URL = row.assets['gec']['href']
    ID = URL.split('/')[-1][:-4]
    # Just omit this b/c will merge w/ original metadata
    #DATETIME = get_datetime(row)

    params = {"id": ID,
              "asset_name": "gec",
              "asset_media_type": "image/tiff; application=geotiff; profile=cloud-optimized",
              "asset_roles": ["data"],
                #"collection": "umbra-sar", # leave out for now (needs a 'self' link)
                #"datetime": DATETIME,
                #"asset_media_type": "auto",
                "with_proj": "true",
                "with_raster": "true",
                "with_eo": "false",
                #"max_size": "1024",
                "geometry_densify": "0",
                "geometry_precision": "-1",
                "url": URL}
    r = requests.get(baseurl, params=params)
    return r.json()

stac = create_stac_item(gf.iloc[10])

In [35]:
# Generate all the STAC ITEMS
async def create_stac_async(row):
    response = await asyncio.to_thread(create_stac_item, row)
    return response

stac_items = await asyncio.gather(*[create_stac_async(row) for i,row in gf1.iterrows()])


In [36]:
# Save as a single item collection
#itemCollection = pystac.ItemCollection(stac_items)
#itemCollection.save_object('panama_timeseries_riostac.geojson')

In [37]:
#stac_items[0]['']

In [38]:
# Merge with original STAC
# Really just want additional properties
#stac_items[0]['properties'].update(orig_items[0].properties)

In [39]:
# new = set(stac_items[1]['properties'].keys())
# orig = set(orig_items[1].properties.keys())
# new.difference(orig)

In [40]:
# orig.difference(new)

In [41]:
#new.intersection(orig) # only one in common is 'datetime'! (which is None since we have start/end instead)

In [42]:
# Add original properites to all the items
for i in range(len(stac_items)):
    stac_items[i]['properties'].update(orig_items[i].properties)

In [43]:
publish_items = [pystac.Item.from_dict(x) for x in stac_items]

In [44]:
publish_items[0] # NOTE: no links at all!

<Item id=2025-01-13-03-36-19_UMBRA-09_GEC>

In [45]:
# Alternative save with a different layout
# NOTE: to *browse* by year would actually have to save annual subcatalogs (or collections?)
#from pystac.layout import TemplateLayoutStrategy
catalog = pystac.Catalog(id='panama-canal', description='Umbra Open SAR Data over Panama Canal')
catalog.add_items(publish_items)

strategy = TemplateLayoutStrategy(item_template="${year}")
#catalog.normalize_hrefs('panama-canal-byyear', strategy=strategy)
catalog.normalize_hrefs('./panama-canal-gec', strategy=strategy)

In [46]:
catalog.save(catalog_type=pystac.CatalogType.SELF_CONTAINED)

In [47]:
# Also save single, consolidated item collection
itemCollection = pystac.ItemCollection(publish_items)
itemCollection.save_object('panama-canal-gec.geojson')